# Notebook responsável por alguns aspectos de modelagem OneToOne / OneToFew Linking (usando MongoEngine)

## Bibliotecas importantes

In [1]:
from mongoengine import *
import pprint
import numpy as np
import pandas as pd
import unidecode

# para geração de dados fake
from faker import Faker


## Classes úteis

In [8]:
class Endereco (Document):
    logradouro = StringField(max_length=200, required=True)
    numero = StringField(max_length=10, required=True)
    bairro = StringField(max_length=50, required=True)
    cidade = StringField(max_length=200, required=True)
    uf = StringField(max_length=50, required=True)
    cep = StringField(max_length=10, required=True)
    
    def __str__(self):
        return f"{self.pk}, {self.logradouro}, {self.numero}"
    
    
class Pessoa (Document):
    cpf = StringField(max_length=20, required=True)
    primeiro_nome = StringField(max_length=200, required=True)
    ultimo_nome = StringField(max_length=400, required=True)
    idade = IntField(min_value=0, max_value=120, required=True)
    data_nascimento = StringField(max_length=20, required=True)
    email = EmailField(max_length=50)
    telefones = ListField(StringField(max_length=50), max_length=4)
    formacoes = ListField(StringField(max_length=50), max_length=4)
    
    # Referência (linking) para um objeto da classe pessoa
    # ref.:https://mongoengine-odm.readthedocs.io/apireference.html#documents
    #endereco = ReferenceField(Endereco, reverse_delete_rule=DENY)
    enderecos = ListField(ReferenceField(Endereco, reverse_delete_rule=DENY))
    
    def __str__(self):
        return f"{self.pk}, {self.cpf}, {self.primeiro_nome}, Endereço: {self.enderecos}"


## Funções uteis

In [9]:
def criar_endereco():
    
    logradouro = fake.street_name()
    numero = fake.building_number()
    bairro = fake.bairro()
    cidade = fake.city()
    uf = fake.administrative_unit()
    cep = fake.postcode()

    endereco = Endereco(logradouro=logradouro, numero=numero, bairro=bairro, cidade=cidade, uf=uf, cep=cep)
    return endereco

In [10]:
def criar_pessoa():
    
    cpf = fake.cpf()
    primeiro_nome = fake.first_name()
    ultimo_nome = fake.last_name()
    idade = np.random.randint(18, 50)
    data_nascimento = fake.date()
    email = f'{cpf}@{fake.free_email_domain()}'
    
    # criando os telefones
    telefones = []
    n = np.random.randint(1, 4)
    for i in range(n):
        telefones.append(fake.msisdn())
    
    # criando as formações
    formacoes = []
    n = np.random.randint(1, 4)
    for i in range(n):
        formacoes.append(fake.job())
    
    enderecos = []
    n = np.random.randint(1, 4)
    for i in range(n):
        enderecos.append(criar_endereco())

    
    pessoa = Pessoa(cpf=cpf, primeiro_nome=primeiro_nome, 
                    ultimo_nome=ultimo_nome, idade=idade, 
                    data_nascimento=data_nascimento, 
                    email=email, telefones=telefones, 
                    formacoes=formacoes, enderecos=enderecos)
    return pessoa

## Configurações

In [11]:
pp = pprint.PrettyPrinter(compact=True)
# gerando dados fictícios em português do Brasil
fake = Faker(['pt_BR'])

## Inserção de vários documentos

### Conectando ao MongoDB

In [12]:
connect('onetomany_linking')

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=3)

### Exemplo - Criação de vários documentos compostos

In [13]:
lista_pessoas = []

for i in range(10):
    lista_pessoas.append(criar_pessoa())
    
pp.pprint(lista_pessoas)

[<Pessoa: None, 483.967.520-10, Olivia, Endereço: [<Endereco: Morro Nascimento, 821>, <Endereco: Pátio de Duarte, 180>]>,
 <Pessoa: None, 670.381.295-03, Davi, Endereço: [<Endereco: Trevo Sarah Ramos, 83>, <Endereco: Setor de Santos, 6>]>,
 <Pessoa: None, 870.139.625-03, Valentina, Endereço: [<Endereco: Residencial Giovanna Lopes, 43>]>,
 <Pessoa: None, 045.381.627-44, Julia, Endereço: [<Endereco: Sítio Sofia Ferreira, 69>]>,
 <Pessoa: None, 027.349.861-40, Luiza, Endereço: [<Endereco: Ladeira Maria Sophia Araújo, 59>, <Endereco: Favela Souza, 90>]>,
 <Pessoa: None, 863.195.270-21, Melissa, Endereço: [<Endereco: Estação da Mata, 98>, <Endereco: Sítio Castro, 6>, <Endereco: Morro de Campos, 66>]>,
 <Pessoa: None, 281.530.974-23, Júlia, Endereço: [<Endereco: Vale de Correia, 74>, <Endereco: Favela de Porto, 8>, <Endereco: Vila Benício Aragão, 57>]>,
 <Pessoa: None, 045.213.789-60, Sofia, Endereço: [<Endereco: Residencial Eduardo Santos, 612>, <Endereco: Lago da Cunha, 80>]>,
 <Pessoa: No

In [14]:
pp.pprint(lista_pessoas[0].pk)

None


### Inserindo os documentos no banco de dados

In [18]:
# inserindo 1 documento apenas
for e in lista_pessoas[0].enderecos:
    e.save()
lista_pessoas[0].save()

<Pessoa: 625f5015cc40c07d0d148255, 483.967.520-10, Olivia, Endereço: [<Endereco: Morro Nascimento, 821>, <Endereco: Pátio de Duarte, 180>]>

In [19]:
pp.pprint(lista_pessoas[0].pk)

ObjectId('625f5015cc40c07d0d148255')


In [22]:
pp.pprint(lista_pessoas[0].enderecos)

[<Endereco: Morro Nascimento, 821>, <Endereco: Pátio de Duarte, 180>]


In [23]:
# inserindo todos os outros documentos
for p in lista_pessoas[1:]:
    for e in p.enderecos:
        e.save()
    p.save()

## Recuperando os documentos inseridos

#### Recuperar todos

In [24]:
for p in Pessoa.objects:
    pp.pprint(p.id)

ObjectId('625f5015cc40c07d0d148255')
ObjectId('625f5050cc40c07d0d148258')
ObjectId('625f5050cc40c07d0d14825a')
ObjectId('625f5050cc40c07d0d14825c')
ObjectId('625f5050cc40c07d0d14825f')
ObjectId('625f5050cc40c07d0d148263')
ObjectId('625f5050cc40c07d0d148267')
ObjectId('625f5050cc40c07d0d14826a')
ObjectId('625f5050cc40c07d0d14826e')
ObjectId('625f5050cc40c07d0d148271')


In [25]:
pessoa = Pessoa.objects.first()
pp.pprint(pessoa)

<Pessoa: 625f5015cc40c07d0d148255, 483.967.520-10, Olivia, Endereço: [<Endereco: Morro Nascimento, 821>, <Endereco: Pátio de Duarte, 180>]>


In [27]:
pp.pprint(pessoa.enderecos[0].pk)

ObjectId('625f5015cc40c07d0d148253')


In [28]:
endereco = Endereco.objects(id='625f5015cc40c07d0d148253').first()
endereco.delete()

OperationError: Could not delete document (Pessoa.enderecos refers to it)

In [ ]:
Pessoa.objects(id='625f48b63a1f265e4af5816b')